In [1]:
import tensorflow as tf
tf.test.gpu_device_name()
#If the GPU is enabled, it will give the following output: '/device:GPU:0'

2023-08-15 09:49:39.259643: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


''

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Aug 15 14:30:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    50W / 400W |    707MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [6]:
%pip install textblob

  Using cached textblob-0.17.1-py2.py3-none-any.whl (636 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string

import nltk as nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from textblob import TextBlob


In [2]:
%pip install ktrain

  Using cached ktrain-0.37.6.tar.gz (25.3 MB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 34.1 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.4 MB/s eta 0:00:0000:0100:01
  Using cached fastprogress-1.0.3-py3-none-any.whl (12 kB)
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py) ... done
  Using cached jieba-0.42.1.tar.gz (19.2 MB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 15.1 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 6.3 MB/s eta 0:00:00
  Using cached syntok-1.4.4-py3-none-any.whl (24 kB)
  Using cached tika-2.6.0.tar.gz (27 kB)
  Preparing metadata (setup.py) ... done
  Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.1 MB/s eta 0:00:0000:01
  Using cach

In [3]:
import ktrain
print('Ktrain Version: ' + ktrain.__version__)

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Ktrain Version: 0.37.6


In [4]:
#%pip install bokeh
import bokeh
print('Bokeh Version: ' + bokeh.__version__)

Bokeh Version: 3.2.2


In [5]:
#%pip install pyarrow

import pyarrow as pa
import pyarrow.parquet as pq

In [6]:
#%pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [7]:
import sklearn
import os
import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn import metrics

from joblib import dump, load
from joblib import Parallel, delayed

In [8]:
#%pip install transformers
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [11]:
#%pip install pandarallel
import multiprocessing

num_processors = multiprocessing.cpu_count()
print(f'Available CPUs: {num_processors}')

import pandarallel
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False)

Available CPUs: 8
INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [10]:

output_file_parquet = pq.read_table('/content/gdrive/MyDrive/Colab dataset/df_filtered.parquet')
df_relevant_articles = output_file_parquet.to_pandas()


Mounted at /content/gdrive


In [11]:
sample_df = df_relevant_articles.sample(frac=0.7, replace=True, random_state=1)

sample_df.head()

,date,processed_text
128037,2023-01-11,vymo release comprehensive guide sale engageme...
5192,2023-06-28,cvpr reveals top five trend computer vision cv...
50057,2023-05-09,aireview ai assistant validate stay booking pr...
109259,2023-03-08,activefence acquires rewire build next generat...
73349,2023-05-31,china warns artificial intelligence risk call ...


In [ ]:
texts = sample_df['processed_text']

In [35]:
df_relevant_articles['date'] = pd.to_datetime(df_relevant_articles['date'])

# Filter and create separate DataFrames for each year
df_2020 = df_relevant_articles[df_relevant_articles['date'].dt.year == 2020].copy()
df_2021 = df_relevant_articles[df_relevant_articles['date'].dt.year == 2021].copy()
df_2022 = df_relevant_articles[df_relevant_articles['date'].dt.year == 2022].copy()
df_2023 = df_relevant_articles[df_relevant_articles['date'].dt.year == 2023].copy()


In [38]:
df_2020.to_parquet('/content/gdrive/MyDrive/Colab dataset/df_2020.parquet', index=False)
df_2021.to_parquet('/content/gdrive/MyDrive/Colab dataset/df_2021.parquet', index=False)
df_2022.to_parquet('/content/gdrive/MyDrive/Colab dataset/df_2022.parquet', index=False)
df_2023.to_parquet('/content/gdrive/MyDrive/Colab dataset/df_2023.parquet', index=False)

### Transformer from Hugging Face (Fine-tuned DistilRoBERTa)
https://huggingface.co/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis


In [15]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

In [12]:
from tqdm import tqdm
import torch.nn.functional as F

def sentiment_trans(text):
    token=tokenizer(text, padding='max_length', max_length=512, truncation=True, return_tensors='pt')
    output=model(**token)
    probs=F.softmax(output.logits)
    probs=probs.detach().numpy()
    return probs

#### Test

In [25]:
tqdm.pandas()
sample_df_100 = sample_df.sample(100)
sample_df_100['sentiment_prob_trans'] = sample_df_100['processed_text'].progress_apply(sentiment_trans)

  0%|          | 0/100 [00:00<?, ?it/s]<ipython-input-24-3c20489242d6>:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs=F.softmax(output.logits)
100%|██████████| 100/100 [00:44<00:00,  2.23it/s]


In [26]:
sample_df_100['sentiment_label_trans'] = sample_df_100['sentiment_prob_trans'].apply(lambda x: np.argmax(x[0]))


In [27]:
sample_df_100

,date,processed_text,sentiment_prob_bert,sentiment_label_bert
123961,2023-02-09,microsoft bakes chatgptlike tech search engine...,"[[0.0001755292, 0.9994842, 0.00034032855]]",1
101139,2023-03-09,move artificial intelligence scientist announc...,"[[6.266023e-05, 0.99980575, 0.00013158149]]",1
43484,2021-11-23,artificial intelligence school demand realworl...,"[[6.910905e-05, 0.99900824, 0.0009227586]]",1
152251,2023-02-23,vsblty deploys trio updated platform capabilit...,"[[0.00024406149, 0.023455506, 0.9763005]]",2
81718,2021-11-27,artificial intelligence technology solution in...,"[[0.00082693924, 0.0056239697, 0.99354917]]",2
...,...,...,...,...
46564,2023-06-15,accenture plan pump 3 billion data ai three ye...,"[[0.00046675364, 0.58301663, 0.4165166]]",1
10879,2023-02-22,nexxiot onboards chief data officer accelerate...,"[[0.0003498038, 0.014826649, 0.9848236]]",2
117833,2020-01-19,government exploring use ai tackle social medi...,"[[9.3325645e-05, 0.9998155, 9.114073e-05]]",1
14473,2020-03-18,specialized processor accelerate endpoint ai w...,"[[9.7373435e-05, 0.99955136, 0.00035128597]]",1


#### Apply to 2020

In [39]:
tqdm.pandas()
df_2020['sentiment_prob_trans'] = df_2020['processed_text'].progress_apply(sentiment_trans)

  0%|          | 0/23273 [00:00<?, ?it/s]<ipython-input-29-4acf5c1af593>:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs=F.softmax(output.logits)
100%|██████████| 23273/23273 [2:20:35<00:00,  2.76it/s]


In [41]:
df_2020['sentiment_label_trans'] = df_2020['sentiment_prob_trans'].apply(lambda x: np.argmax(x[0]))


In [44]:
df_2020.to_csv('/content/gdrive/MyDrive/Colab dataset/df_2020.csv', index=False)
df_2020.head()

,date,processed_text,sentiment_prob_trans,sentiment_label_trans
0,2020-02-27,child autism saw learning social skill boosted...,"[[0.00039809354, 0.06718684, 0.932415]]",2
4,2020-06-14,artificial intelligence behavioral mental heal...,"[[0.0003117534, 0.009448456, 0.99023974]]",2
5,2020-07-10,ai machine learning market 2020 expected reach...,"[[0.00046173323, 0.091090694, 0.9084475]]",2
6,2020-03-16,according latest report machine learning cours...,"[[0.00021497608, 0.00036592176, 0.9994191]]",2
7,2020-02-19,eu imposes sweeping regulation facial recognit...,"[[0.00013172136, 0.99969137, 0.00017702393]]",1


#### Apply to 2021

In [16]:
output_file_parquet = pq.read_table('/content/gdrive/MyDrive/Colab dataset/df_2021.parquet')
df_2021 = output_file_parquet.to_pandas()

In [17]:
tqdm.pandas()
df_2021['sentiment_prob_trans'] = df_2021['processed_text'].progress_apply(sentiment_trans)

  0%|          | 0/29765 [00:00<?, ?it/s]<ipython-input-15-4acf5c1af593>:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs=F.softmax(output.logits)
100%|██████████| 29765/29765 [4:04:00<00:00,  2.03it/s]


In [18]:
df_2021['sentiment_label_trans'] = df_2021['sentiment_prob_trans'].apply(lambda x: np.argmax(x[0]))


In [19]:
df_2021.to_csv('/content/gdrive/MyDrive/Colab dataset/df_2021.csv', index=False)
df_2021.head()

,date,processed_text,sentiment_prob_trans,sentiment_label_trans
0,2021-11-24,college technology launch ai retail lab cougar...,"[[8.414829e-05, 0.9998129, 0.00010296759]]",1
1,2021-02-25,multicoreware inc becomes cevas trusted partne...,"[[0.00020204605, 0.94319, 0.056608006]]",1
2,2021-04-03,calling startup ai space africacom calling sta...,"[[8.081454e-05, 0.9990225, 0.0008966643]]",1
3,2021-03-11,artificial intelligence accounting market valu...,"[[9.6110525e-05, 0.9997881, 0.000115855386]]",1
4,2021-05-12,myelin foundry funding ai startup myelin found...,"[[0.00011249224, 0.99797386, 0.0019135429]]",1


#### Apply to 2022

In [17]:
#output_file_parquet = pq.read_table('/content/gdrive/MyDrive/Colab dataset/df_2022.parquet')
#df_2022 = output_file_parquet.to_pandas()
df_2022 = pd.read_parquet('df_2022.parquet')

In [18]:
tqdm.pandas()
df_2022['sentiment_prob_trans'] = df_2022['processed_text'].progress_apply(sentiment_trans)

  0%|          | 0/36473 [00:00<?, ?it/s]/var/folders/50/cr_04dgs14bf0wgx8frly6s40000gn/T/ipykernel_87805/1042620942.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs=F.softmax(output.logits)
100%|██████████| 36473/36473 [4:48:30<00:00,  2.11it/s]   


In [19]:
df_2022['sentiment_label_trans'] = df_2022['sentiment_prob_trans'].apply(lambda x: np.argmax(x[0]))


In [20]:
#df_2022.to_csv('/content/gdrive/MyDrive/Colab dataset/df_2022.csv', index=False)
df_2022.to_csv('df_2022_transformer.csv', index=False)

df_2022.head()

,date,processed_text,sentiment_prob_trans,sentiment_label_trans
0,2022-10-06,healthcare artificial intelligence market anal...,"[[7.482795e-05, 0.9983261, 0.0015989639]]",1
1,2022-12-02,paragon semvox volkswagens cariad buy paragon ...,"[[7.687068e-05, 0.99981004, 0.000113123155]]",1
2,2022-06-12,google ai become sentient think like sevenyear...,"[[0.0001862179, 0.99966383, 0.00014987536]]",1
3,2022-02-17,cornell researcher train physical system revol...,"[[0.00036868497, 0.99796176, 0.0016696041]]",1
4,2022-09-21,nvidia jetson orin nano systemonmodules advanc...,"[[0.00056856085, 0.81929946, 0.18013203]]",1


#### Apply to 2023

In [22]:
#output_file_parquet = pq.read_table('/content/gdrive/MyDrive/Colab dataset/df_2023.parquet')
#df_2023 = output_file_parquet.to_pandas()
df_2023 = pd.read_parquet('df_2023.parquet')

In [24]:
#data for 2023 is too big, have to sample
df_2023_sample = df_2023.sample(frac=0.6, replace=False, random_state=1)


In [25]:
tqdm.pandas()
df_2023_sample['sentiment_prob_trans'] = df_2023_sample['processed_text'].progress_apply(sentiment_trans)

  0%|          | 0/40679 [00:00<?, ?it/s]/var/folders/50/cr_04dgs14bf0wgx8frly6s40000gn/T/ipykernel_87805/1042620942.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs=F.softmax(output.logits)
100%|██████████| 40679/40679 [5:17:19<00:00,  2.14it/s]   


In [26]:
df_2023_sample['sentiment_label_trans'] = df_2023_sample['sentiment_prob_trans'].apply(lambda x: np.argmax(x[0]))


In [28]:
#df_2023.to_csv('/content/gdrive/MyDrive/Colab dataset/df_2023.csv', index=False)
df_2023_sample.to_csv('df_2023_transformer.csv', index=False)

df_2023_sample.head()

,date,processed_text,sentiment_prob_trans,sentiment_label_trans
51359,2023-03-28,nvidia corp nvidia show new research using ai ...,"[[0.00017280378, 0.0019943917, 0.99783283]]",2
61271,2023-06-07,rishi sunak go washington ukraine economy ai a...,"[[9.963714e-05, 0.99978477, 0.00011565144]]",1
6327,2023-06-19,compass uol launch ai cockpit gen ai compass u...,"[[7.042589e-05, 0.9995059, 0.0004237531]]",1
21054,2023-06-22,europe bestselling newspaper face job cut embr...,"[[0.41428712, 0.56546277, 0.020250125]]",1
26515,2023-04-17,global conversational ai market report 2023 in...,"[[0.00018769651, 0.00024475012, 0.99956757]]",2
